In [14]:
!pip install lightgbm

import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

import pandas as pd
import numpy as np


In [73]:
## Sirisha final data frame- Parent to subset data frames
df = pd.read_csv('Final_Data_Without_Social_Det_v7.csv')

In [16]:
df_holdout = pd.read_csv('merged_Holdout_data.csv')

In [74]:
df.shape

(1527904, 82)

In [6]:
df_holdout.shape

(381976, 85)

In [9]:
df_list = list(df.columns)

In [10]:
df_holdout_list = list(df_holdout.columns)

In [12]:
result = [i for i in df_holdout_list if i not in df_list]

In [13]:
result

['cms_frailty_ind',
 'plan_benefit_package_id',
 'consec_tenure_month',
 'cnt_cp_livecall_pmpm_ct',
 'rx_perphy_pmpm_ct',
 'PCP_VISIT_PREV_YR']

In [77]:
social_determinants = pd.read_csv('final_social_v3.csv')

In [78]:
social_determinants.shape

(1527904, 41)

In [79]:
social_determinants = pd.merge(social_determinants, df[['id','preventive_visit_gap_ind']], on = 'id',how = 'left')

In [80]:
social_determinants.shape

(1527904, 42)

In [17]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest

In [18]:
mi_selector = SelectKBest(mutual_info_classif, k='all')

In [25]:
X_mutual = social_determinants.drop(['id','preventive_visit_gap_ind'],axis = 1)
y_mutual = social_determinants[['preventive_visit_gap_ind']]

In [28]:
X_mutual.fillna(0,inplace = True)

In [29]:
mi = mi_selector.fit_transform(X_mutual,y_mutual)

/home/dataminions60/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [33]:
(mi_selector.scores_)

array([0.03298121, 0.03303878, 0.02919948, 0.03371996, 0.03331263,
       0.03259689, 0.03329803, 0.03288427, 0.01268248, 0.03314192,
       0.03319064, 0.03279786, 0.03175582, 0.03252719, 0.00603117,
       0.03230194, 0.0323492 , 0.03319989, 0.03298489, 0.03316561,
       0.0335214 , 0.00752927, 0.03261041, 0.03360831, 0.01010275,
       0.01932195, 0.03308665, 0.03297012, 0.01233568, 0.00777519,
       0.03308105, 0.03365843, 0.03317813, 0.03253617, 0.03256637,
       0.03346671, 0.01228033, 0.03314189, 0.03152586, 0.03265001])

In [36]:
mi_df = pd.DataFrame({'Feature': X_mutual.columns, 'Mutual Information': mi_selector.scores_})
print(mi_df.sort_values(by='Mutual Information', ascending=False))

                         Feature  Mutual Information
3              rwjf_unemploy_pct            0.033720
31   rwjf_insufficient_sleep_pct            0.033658
23                rwjf_other_pcp            0.033608
20           rwjf_dentists_ratio            0.033521
35  rwjf_housing_cost_burden_pct            0.033467
4        rwjf_poor_phy_hlth_days            0.033313
6        rwjf_income_inequ_ratio            0.033298
17          rwjf_drive_alone_pct            0.033200
10            rwjf_age_gt_65_pct            0.033191
32    rwjf_physical_distress_pct            0.033178
19            rwjf_uninsured_pct            0.033166
9                rwjf_population            0.033142
37         rwjf_broadband_access            0.033142
26       rwjf_injury_deaths_rate            0.033087
30    rwjf_social_associate_rate            0.033081
1                  rwjf_pcp_rate            0.033039
18     rwjf_limit_hlthy_food_pct            0.032985
0           rwjf_poor_health_pct            0.

In [24]:
social_determinants.columns

Index(['rwjf_poor_health_pct', 'rwjf_pcp_rate', 'rwjf_preventable_ip_rate',
       'rwjf_unemploy_pct', 'rwjf_poor_phy_hlth_days',
       'rwjf_poor_men_hlth_days', 'rwjf_income_inequ_ratio',
       'rwjf_alcoholic_pct', 'rwjf_mammography_pct', 'rwjf_population',
       'rwjf_age_gt_65_pct', 'rwjf_female_pct', 'rwjf_rural_pct',
       'rwjf_non_english_pct', 'rwjf_diabetes_pct', 'rwjf_men_hlth_prov_ratio',
       'rwjf_median_house_income', 'rwjf_drive_alone_pct',
       'rwjf_limit_hlthy_food_pct', 'rwjf_uninsured_pct',
       'rwjf_dentists_ratio', 'rwjf_air_pollute_density',
       'rwjf_premature_mortality', 'rwjf_other_pcp', 'rwjf_food_env_inx',
       'rwjf_dui_deaths_pct', 'rwjf_injury_deaths_rate',
       'rwjf_severe_housing_pct', 'rwjf_long_commute_alone_pct',
       'rwjf_food_insecurity_pct', 'rwjf_social_associate_rate',
       'rwjf_insufficient_sleep_pct', 'rwjf_physical_distress_pct',
       'rwjf_mental_distress_pct', 'rwjf_home_ownership_pct',
       'rwjf_housing_cos

In [68]:
import statsmodels.formula.api as sm
def vif_cal(input_data, dependent_col,unique_id):
    x_vars=input_data.drop([dependent_col,unique_id], axis=1)
    xvar_names=x_vars.columns
    for i in range(0,xvar_names.shape[0]):
        y=x_vars[xvar_names[i]]
        x=x_vars[xvar_names.drop(xvar_names[i])]
        rsq=sm.ols(formula="y~x", data=x_vars).fit().rsquared
        vif=round(1/(1-rsq),2)
        print (xvar_names[i], " VIF = " , vif)

In [69]:
vif_cal(social_determinants,'preventive_visit_gap_ind','id')

rwjf_poor_health_pct  VIF =  28.78
rwjf_pcp_rate  VIF =  3.71
rwjf_preventable_ip_rate  VIF =  2.13
rwjf_unemploy_pct  VIF =  2.57
rwjf_poor_phy_hlth_days  VIF =  58.74
rwjf_poor_men_hlth_days  VIF =  28.99
rwjf_income_inequ_ratio  VIF =  2.85
rwjf_alcoholic_pct  VIF =  2.66
rwjf_mammography_pct  VIF =  2.48
rwjf_population  VIF =  2.45
rwjf_age_gt_65_pct  VIF =  3.79
rwjf_female_pct  VIF =  1.59
rwjf_rural_pct  VIF =  6.03
rwjf_non_english_pct  VIF =  5.16
rwjf_diabetes_pct  VIF =  2.8
rwjf_men_hlth_prov_ratio  VIF =  1.92
rwjf_median_house_income  VIF =  8.21
rwjf_drive_alone_pct  VIF =  2.56
rwjf_limit_hlthy_food_pct  VIF =  201.27
rwjf_uninsured_pct  VIF =  3.45
rwjf_dentists_ratio  VIF =  3.14
rwjf_air_pollute_density  VIF =  1.73
rwjf_premature_mortality  VIF =  10.28
rwjf_other_pcp  VIF =  2.9
rwjf_food_env_inx  VIF =  935.98
rwjf_dui_deaths_pct  VIF =  1.15
rwjf_injury_deaths_rate  VIF =  4.07
rwjf_severe_housing_pct  VIF =  8.51
rwjf_long_commute_alone_pct  VIF =  3.11
rwjf_fo

In [82]:
##After VIF and Mutual info classification, I have selected below columns for modelling from social determinants
selected_cols = [
'rwjf_unemploy_pct',
'rwjf_insufficient_sleep_pct',
'rwjf_other_pcp',
'rwjf_dentists_ratio',
'rwjf_income_inequ_ratio',
'rwjf_drive_alone_pct',
'rwjf_age_gt_65_pct',
'rwjf_uninsured_pct',
'rwjf_population',
'rwjf_injury_deaths_rate',
'rwjf_social_associate_rate',
'rwjf_pcp_rate',
'rwjf_alcoholic_pct',
'rwjf_female_pct',
'rwjf_healthcare_cost',
'rwjf_diabetes_monitor_pct',
'rwjf_preventable_ip_rate',
'rwjf_dui_deaths_pct',
'rwjf_mammography_pct',
'rwjf_long_commute_alone_pct',
'rwjf_flu_vax',
'id']


In [83]:
social_determinants = social_determinants[selected_cols]

In [84]:
social_determinants.shape

(1527904, 22)

In [85]:
social_determinants.head()

,rwjf_unemploy_pct,rwjf_insufficient_sleep_pct,rwjf_other_pcp,rwjf_dentists_ratio,rwjf_income_inequ_ratio,rwjf_drive_alone_pct,rwjf_age_gt_65_pct,rwjf_uninsured_pct,rwjf_population,rwjf_injury_deaths_rate,...,rwjf_alcoholic_pct,rwjf_female_pct,rwjf_healthcare_cost,rwjf_diabetes_monitor_pct,rwjf_preventable_ip_rate,rwjf_dui_deaths_pct,rwjf_mammography_pct,rwjf_long_commute_alone_pct,rwjf_flu_vax,id
0,0.038508,0.419475,0.000653,0.000179,4.365779,0.810576,0.132866,0.159579,111744.0,76.222466,...,0.149823,0.528834,9750.99,0.863121,52.68,0.224719,0.41,0.486,0.49,1352611
1,0.031306,0.406581,0.000598,0.000486,4.371184,0.806650,0.193761,0.066314,26734.0,88.178060,...,0.166493,0.520236,8749.57,0.881720,32.74,0.318182,0.45,0.318,0.55,299881
2,0.036413,0.410512,0.000684,0.000333,4.540098,0.805270,0.155243,0.076147,54115.0,96.339110,...,0.162364,0.513887,9544.74,0.896078,33.49,0.228571,0.44,0.377,0.56,1191826
3,0.050487,0.424149,0.000896,0.000081,4.440090,0.808773,0.188032,0.081235,24549.0,115.117070,...,0.147714,0.509634,10177.70,0.890449,55.07,0.225806,0.31,0.395,0.37,1772108
4,0.064084,0.447947,0.002416,0.000674,5.869788,0.876243,0.188429,0.074765,35589.0,118.310135,...,0.134688,0.512153,13539.30,0.821641,107.71,0.098039,0.30,0.313,0.34,171691


In [86]:
df = pd.merge(df,social_determinants, on = 'id', how = 'left')

In [87]:
df.shape

(1527904, 103)

In [88]:
df.columns

Index(['id', 'num_of_chronic_conditions', 'preventive_visit_gap_ind',
       'measure_name_lag3', 'dissatisfaction_cnt_2020', 'measure_name_lag2',
       'dissatisfaction_cnt_2021', 'measure_name_lag1',
       'dissatisfaction_cnt_2022', 'PV_PREV_YR',
       ...
       'rwjf_pcp_rate', 'rwjf_alcoholic_pct', 'rwjf_female_pct',
       'rwjf_healthcare_cost', 'rwjf_diabetes_monitor_pct',
       'rwjf_preventable_ip_rate', 'rwjf_dui_deaths_pct',
       'rwjf_mammography_pct', 'rwjf_long_commute_alone_pct', 'rwjf_flu_vax'],
      dtype='object', length=103)

In [89]:
subset_1 = df.copy()

In [90]:
subset_1.drop(['age_Tile_Num','total_ip_health_issues_admit_days_pmpm','nonpar_allowed_pmpm_cost','total_allowed_pmpm_cost','nonpar_clm_ct_pmpm',
               'nonpar_ds_clm','days_since_last_clm','nonpar_net_paid_pmpm_cost','total_net_paid_pmpm_cost','rx_overall_mbr_resp_pmpm_cost',
              'rx_all_tiers_pmpm_ct', 'rx_overall_dist_gpi6_pmpm_ct','login_pmpm_ct',
              'mco_contract_nbr', 'pbp_segment_id', 'total_deduct_pmpm_cost', 'total_interactions_2', 'total_interactions_0',
               'total_interactions_3'],axis = 1, inplace = True)

In [91]:
subset_1.shape

(1527904, 84)

In [92]:
subset_1['num_of_chronic_conditions'] = np.where(subset_1['num_of_chronic_conditions'] == -1, 0, subset_1['num_of_chronic_conditions'])

In [93]:
subset_1.head()

,id,num_of_chronic_conditions,preventive_visit_gap_ind,measure_name_lag3,dissatisfaction_cnt_2020,measure_name_lag2,dissatisfaction_cnt_2021,measure_name_lag1,dissatisfaction_cnt_2022,PV_PREV_YR,...,rwjf_pcp_rate,rwjf_alcoholic_pct,rwjf_female_pct,rwjf_healthcare_cost,rwjf_diabetes_monitor_pct,rwjf_preventable_ip_rate,rwjf_dui_deaths_pct,rwjf_mammography_pct,rwjf_long_commute_alone_pct,rwjf_flu_vax
0,1787899,0,1,4,0,4,1,13,1,0.0,...,0.000588,0.181928,0.511140,11620.61,0.846231,53.29,0.210651,0.40,0.426,0.50
1,1097519,2,0,6,0,5,0,2,0,0.0,...,0.000535,0.189056,0.513011,11554.89,0.868121,58.41,0.261029,0.42,0.447,0.51
2,1284744,0,1,0,0,0,0,0,0,NaN,...,0.000753,0.212348,0.503298,7669.68,0.899506,19.31,0.227273,0.54,0.155,0.41
3,1012315,0,0,2,1,2,1,3,1,NaN,...,0.000689,0.181873,0.518192,11595.93,0.792079,62.51,0.284404,0.46,0.385,0.47
4,279911,0,1,10,3,2,1,2,1,NaN,...,0.000530,0.211616,0.494078,9210.51,0.822439,37.49,0.242798,0.39,0.189,0.39


In [94]:
mapping = {
    '1-Metro': 1,
    '2-Metro': 1,
    '3-Metro': 1,
    '6-Nonmetro': 0,
    '4-Nonmetro': 0,
    '7-Nonmetro': 0,
    '5-Nonmetro	': 0,
    '8-Nonmetro': 0,
    '9-Nonmetro': 0
}

subset_1['rucc_category'] = subset_1['rucc_category'].map(mapping)

In [95]:
subset_1['rucc_category'] = subset_1['rucc_category'].fillna(1)

In [96]:
subset_1.fillna(0, inplace = True)

In [97]:
subset_1.shape

(1527904, 84)

In [98]:
# prompt: one hot encoding for race

# Perform one-hot encoding for the 'race' column
race_encoded = pd.get_dummies(subset_1['race'], prefix='race', drop_first=True, dtype=int)

# Concatenate the one-hot encoded columns with the original DataFrame
subset_1 = pd.concat([subset_1, race_encoded], axis=1)

# Drop the original 'race' column if you no longer need it
subset_1 = subset_1.drop('race', axis=1)

# Print the updated DataFrame to verify the changes
subset_1.head()

,id,num_of_chronic_conditions,preventive_visit_gap_ind,measure_name_lag3,dissatisfaction_cnt_2020,measure_name_lag2,dissatisfaction_cnt_2021,measure_name_lag1,dissatisfaction_cnt_2022,PV_PREV_YR,...,rwjf_mammography_pct,rwjf_long_commute_alone_pct,rwjf_flu_vax,race_BLACK,race_HISPANIC,race_N AMERICAN NATIVE,race_OTHER,race_UNKNOWN,race_Unknown,race_WHITE
0,1787899,0,1,4,0,4,1,13,1,0.0,...,0.40,0.426,0.50,0,0,0,0,0,1,0
1,1097519,2,0,6,0,5,0,2,0,0.0,...,0.42,0.447,0.51,0,0,0,0,0,0,1
2,1284744,0,1,0,0,0,0,0,0,0.0,...,0.54,0.155,0.41,0,0,0,0,0,1,0
3,1012315,0,0,2,1,2,1,3,1,0.0,...,0.46,0.385,0.47,0,0,0,0,0,0,1
4,279911,0,1,10,3,2,1,2,1,0.0,...,0.39,0.189,0.39,0,0,0,0,0,1,0


In [99]:
subset_1.replace('Unknown', "UNKNOWN", inplace=True)

In [100]:
# Replace 'Florida' with 'Gulf South' in the 'region' column
subset_1['region'] = subset_1['region'].replace('Florida', 'Gulf South')
subset_1['region'] = np.where(subset_1['region'] == 0, 'Southeast', subset_1['region'])

In [101]:
# Perform one-hot encoding for the 'region' column
region_encoded = pd.get_dummies(subset_1['region'], prefix='region', drop_first=True, dtype=int)

# Concatenate the one-hot encoded columns with the original DataFrame
subset_1 = pd.concat([subset_1, region_encoded], axis=1)

# Drop the original 'region' column if you no longer need it
subset_1 = subset_1.drop('region', axis=1)
subset_1.columns


Index(['id', 'num_of_chronic_conditions', 'preventive_visit_gap_ind',
       'measure_name_lag3', 'dissatisfaction_cnt_2020', 'measure_name_lag2',
       'dissatisfaction_cnt_2021', 'measure_name_lag1',
       'dissatisfaction_cnt_2022', 'PV_PREV_YR', 'all_mm_tenure',
       'dual_eligible_ind', 'disabled_ind', 'lis_ind', 'tenure_band_grouped',
       'sex_cd', 'age', 'veteran_ind', 'nonpar_coins_pmpm_cost',
       'total_coins_pmpm_cost', 'nonpar_cob_paid_pmpm_cost',
       'total_cob_paid_pmpm_cost', 'nonpar_copay_pmpm_cost',
       'total_copay_pmpm_cost', 'nonpar_deduct_pmpm_cost',
       'nonpar_mbr_resp_pmpm_cost', 'total_mbr_resp_pmpm_cost',
       'rx_days_since_last_script', 'rx_overall_coins_copay_cost',
       'rx_overall_deduct_pmpm_cost', 'rx_overall_pmpm_cost',
       'rx_overall_pmpm_ct', 'rx_pharmacies_pmpm_ct', 'login_count_0',
       'login_count_1', 'days_since_last_login', 'lag_quarter',
       'lag_six_months', 'cnt_cp_emails_pmpm_ct', 'cnt_cp_print_pmpm_ct',
     

In [102]:
# prompt: train test split target is preventive_visit_gap_ind

from sklearn.model_selection import train_test_split


# Define your features (X) and target (y)
X = subset_1.drop(['preventive_visit_gap_ind','id'], axis=1)
y = subset_1['preventive_visit_gap_ind']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Print the shapes of the resulting datasets
#Round 1
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (1069532, 95)
X_test shape: (458372, 95)
y_train shape: (1069532,)
y_test shape: (458372,)


In [103]:
X_train_1 = X_train.copy()
X_test_1 = X_test.copy()

In [104]:
#Round 2
X_train_1.drop(['atlas_ffrpth14', 'atlas_povertyallagespct','login_count_1','login_count_0','nonpar_coins_pmpm_cost','nonpar_cob_paid_pmpm_cost','nonpar_deduct_pmpm_cost','total_coins_pmpm_cost'], axis = 1, inplace = True)
X_test_1.drop(['atlas_ffrpth14', 'atlas_povertyallagespct', 'login_count_1','login_count_0','nonpar_coins_pmpm_cost','nonpar_cob_paid_pmpm_cost','nonpar_deduct_pmpm_cost','total_coins_pmpm_cost'], axis = 1, inplace = True)


In [105]:
X_train_1.shape

(1069532, 87)

In [106]:
# prompt: create a function to calculate accuracy, confusion matrix, auc score

from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

def evaluate_model(y_true, y_pred, y_prob=None):
  """
  Calculates accuracy, confusion matrix, and AUC score for a model.

  Args:
    y_true: True labels.
    y_pred: Predicted labels.
    y_prob: Predicted probabilities (optional for AUC score).

  Returns:
    A dictionary containing accuracy, confusion matrix, and AUC score.
  """
  results = {}
  results['test_accuracy'] = accuracy_score(y_true, y_pred)
  results['confusion_matrix'] = confusion_matrix(y_true, y_pred)
  if y_prob is not None:
    results['auc'] = roc_auc_score(y_true, y_prob)
  return results


In [65]:
import time

# Create a LightGBM classifier
lgb_classifier = lgb.LGBMClassifier(random_state=42)


# Define the parameter grid for LightGBM
param_grid_lgb = {
    'n_estimators': [ 50,60,70],
    'learning_rate': [0.01, 0.1],
    'max_depth': [10,15],
    'num_leaves': [100, 200],
    'class_weight': ['balanced']
}

# Create a GridSearchCV object for LightGBM
grid_search_lgb = GridSearchCV(estimator=lgb_classifier, param_grid=param_grid_lgb, cv=5, scoring='roc_auc')
start_time = time.time()

# Fit the grid search to the training data
grid_search_lgb.fit(X_train_1, y_train)

end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 385080, number of negative: 470545
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.087696 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8821
[LightGBM] [Info] Number of data points in the train set: 855625, number of used features: 83
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.450057 -> initscore=-0.200440
[LightGBM] [Info] Start training from score -0.200440
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 385080, number of negative: 470545
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.091470 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `fo

In [66]:
# Print the best parameters and the best score
print("Best parameters (LightGBM):", grid_search_lgb.best_params_)
print("Best score (LightGBM):", grid_search_lgb.best_score_)
# Get the best model from the grid search
best_lgb_model = grid_search_lgb.best_estimator_


Best parameters (LightGBM): {'learning_rate': 0.1, 'max_depth': 15, 'n_estimators': 70, 'num_leaves': 30}
Best score (LightGBM): 0.7262549747163448


In [67]:
# Make predictions on the test set using the best model
y_pred_lgb = best_lgb_model.predict(X_test_1)
best_lgb_model
params_importance = {
    'n_estimators': 70,
    'learning_rate': 0.1,
    'max_depth': 15,
    'num_leaves': 30
}

In [46]:
#y_pred_lgb_prob = best_lgb_model.predict_proba(X_test_1)[:, 1] 

In [53]:
#threshold = 0.4
#y_pred_lgb_1 = (y_pred_lgb_prob >= threshold).astype(int)

In [69]:
# Evaluate the model
lgb_results = evaluate_model(y_test, y_pred_lgb)
# Print the evaluation results
print("LightGBM Results:")
print("Accuracy:", lgb_results['test_accuracy'])
print("Confusion Matrix:\n", lgb_results['confusion_matrix'])
if 'auc' in lgb_results:
    print("AUC:", lgb_results['auc'])

roc_auc_score(y_test, y_pred_lgb)


LightGBM Results:
Accuracy: 0.6688017592697634
Confusion Matrix:
 [[204965  47697]
 [104115 101595]]


0.6525484896988949

In [70]:
train_data = lgb.Dataset(X_train_1, label=y_train)
model_importance = lgb.train(params_importance, train_data, num_boost_round=100)

y_pred = np.round(model_importance.predict(X_test_1))
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

importance = model_importance.feature_importance()
feature_names = X_test_1.columns

feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)

print(feature_importance_df)

/home/dataminions60/.local/lib/python3.8/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.475661 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8857
[LightGBM] [Info] Number of data points in the train set: 1069532, number of used features: 83
[LightGBM] [Info] Start training from score 0.450057
Accuracy: 0.6692
                        Feature  Importance
0     num_of_chronic_conditions         104
62           rwjf_age_gt_65_pct          82
57  rwjf_insufficient_sleep_pct          74
56            rwjf_unemploy_pct          73
65      rwjf_injury_deaths_rate          66
..                          ...         ...
17               pbp_segment_id           0
42        channel_DMS Telesales           0
21       total_deduct_pmpm_cost           0
39         total_interactions_2           0
16         

In [71]:
feature_importance_df.head(30)

,Feature,Importance
0,num_of_chronic_conditions,104
62,rwjf_age_gt_65_pct,82
57,rwjf_insufficient_sleep_pct,74
56,rwjf_unemploy_pct,73
65,rwjf_injury_deaths_rate,66
61,rwjf_drive_alone_pct,64
64,rwjf_population,61
28,rx_overall_pmpm_ct,60
5,measure_name_lag1,56
66,rwjf_social_associate_rate,56


In [72]:
feature_importance_df.tail(30)

,Feature,Importance
81,region_South Central,6
11,lis_ind,6
12,tenure_band_grouped,6
31,lag_quarter,6
29,rx_pharmacies_pmpm_ct,6
70,race_N AMERICAN NATIVE,5
77,region_Midwest,4
27,rx_overall_pmpm_cost,4
55,dcsi_score,4
76,region_Intermountain,3


In [ ]:
mco_contract_nbr, pbp_segment_id, total_deduct_pmpm_cost, total_interactions_2, total_interactions_0,total_interactions_3